In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,TimeDistributed,GRU,SimpleRNN
from keras.preprocessing.sequence import TimeseriesGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
label_cols=[]

In [ ]:
articles_df=pd.read_csv(r'articles.csv')
customers_df=pd.read_csv(r'customers.csv')
transcations_df=pd.read_csv(r'transactions_train.csv')

In [ ]:
articles_df.head()

In [ ]:
articles_df.info()

In [ ]:
customers_df.head()

In [ ]:
customers_df.info()

In [ ]:
transcations_df.head()

In [ ]:
transcations_df.head()

In [ ]:
def missing_data(func_df):
    total=func_df.isnull().sum().sort_values(ascending=False)
    percent=(func_df.isnull().sum()/func_df.isnull().count()*100).sort_values(ascending=False)
    return pd.concat([total,percent],axis=1,keys=['Total','Percent'])

In [ ]:
missing_data(articles_df)

In [ ]:
missing_data(customers_df)

In [ ]:
missing_data(transcations_df)

In [ ]:
def unique_values(func_df):
    total=func_df.count()
    tt=pd.DataFrame(total)
    tt.columns=['Total']
    uniques=[]
    for col in func_df.columns:
        unique=func_df[col].nunique()
        uniques.append(unique)
    tt['Uniques']=uniques
    return tt

In [ ]:
unique_values(articles_df)

In [ ]:
unique_values(customers_df)

In [ ]:
unique_values(transcations_df)

### Here i am dropping this columns because it has the column which represents in the number format in simple words to reduce multicollinearity
### Eg product_name column has product_name which is same 

In [ ]:
cols=['prod_name','product_type_name','graphical_appearance_name','colour_group_name','perceived_colour_value_name',
      'department_name','index_name','index_group_name','section_name','garment_group_name']
articles_df.drop(columns=cols,axis=1,inplace=True)

In [ ]:
articles_df.head()

In [ ]:
label_encoder = preprocessing.LabelEncoder()
cols=['product_group_name','perceived_colour_master_name','index_code']
for col in cols:
    label_cols.append(col)
    articles_df[col]= label_encoder.fit_transform(articles_df[col]) 

### I will be dropping detail_desc column 

In [ ]:
articles_df.drop(['detail_desc'],axis=1,inplace=True)

In [ ]:
articles_df

In [ ]:
customers_df.head()

In [ ]:
customers_df.shape

In [ ]:
customers_df['fashion_news_frequency'].value_counts()

In [ ]:
print("The precentage of null value in FN column is {}%".format(round(((customers_df['FN'].isna().sum()/customers_df.shape[0])*100),2)))
print("The precentage of null value in Active column is {}%".format(round(((customers_df['Active'].isna().sum()/customers_df.shape[0])*100),2)))

### I am dropping the FN and Active column because it has more than 65 % null values 

In [ ]:
customers_df.drop(['FN','Active'],axis=1,inplace=True)

In [ ]:
customers_df['club_member_status'].isna().sum()

In [ ]:
customers_df['club_member_status'].fillna('ACTIVE',axis=0,inplace=True)

In [ ]:
customers_df['club_member_status'].isna().sum()

In [ ]:
customers_df['fashion_news_frequency'].replace(to_replace='NONE',value='None',inplace=True)

In [ ]:
customers_df['fashion_news_frequency'].value_counts()

In [ ]:
customers_df['postal_code'].nunique()

### Since we has so many unique values in postal_code so  i will be dropping this column

In [ ]:
customers_df.drop(['postal_code'],axis=1,inplace=True)

In [ ]:
customers_df.dropna(axis=0,inplace=True)

In [ ]:
label_encoder = preprocessing.LabelEncoder()
cols=['club_member_status','fashion_news_frequency']
for col in cols:
    label_cols.append(col)
    customers_df[col]= label_encoder.fit_transform(customers_df[col]) 

In [ ]:
customers_df.head()

In [ ]:
transcations_df.head()

### There is no null values in transcations 

In [ ]:
transcations_df['t_dat']=pd.to_datetime(transcations_df["t_dat"])

In [ ]:
transcations_df.tail()

In [ ]:
transcations_df.shape

### Here i have selected only the transcations after the year 2020

In [ ]:
transcations_df=transcations_df[transcations_df['t_dat']>'2020-01-01'].copy()
transcations_df.shape

In [ ]:
transcations_df.head()

In [ ]:
inner_merged=pd.merge(transcations_df,articles_df)#inner merge 

In [ ]:
inner_merged.shape

In [ ]:
inner_merged.head()

In [ ]:
final_merged=pd.merge(inner_merged,customers_df)

In [ ]:
final_merged.shape

In [ ]:
final_merged.head()

### We dont need article_id and customer_id anymore so we will be dropping that too

In [ ]:
final_merged.drop(['customer_id','article_id'],axis=1,inplace=True)

In [ ]:
int_cols=[]
total_columms=final_merged.columns
for col in total_columms:
    if col not in label_cols:
        int_cols.append(col)

In [ ]:
final_merged[int_cols]

In [ ]:
final_merged.columns.intersection(label_cols)

In [ ]:
final_merged.to_csv('cleaned_h_m_dataset',encoding='utf-8',index=False)

In [ ]:
final_merged=pd.read_csv(r'cleaned_h_m_dataset')

In [ ]:
final_merged.head()

In [ ]:
final_merged.t_dat.max()

In [ ]:
df_train=final_merged[(final_merged['t_dat']>'2020-08-28') & (final_merged['t_dat']<'2020-09-20')].copy()
df_test=final_merged[final_merged['t_dat']>='2020-09-20'].copy()

In [ ]:
df_train.t_dat.max()

In [ ]:
df_train.drop(['t_dat'],axis=1,inplace=True)
df_test.drop(['t_dat'],axis=1,inplace=True)

In [ ]:
cols=df_train.columns

In [ ]:
df_product_code_train=df_train.copy()
scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=cols)
df_test=pd.DataFrame(scaler.transform(df_test),columns=cols)
scaler_pred=MinMaxScaler()
df_product_code=pd.DataFrame(scaler_pred.fit_transform(df_product_code_train.product_code.values.reshape(-1,1)))

In [ ]:
y_train=df_train['product_code'].tolist()
y_test=df_test['product_code'].tolist()
df_train.drop(['product_code'],axis=1,inplace=True)
df_test.drop(['product_code'],axis=1,inplace=True)
x_train=df_train.to_numpy().tolist()
x_test=df_test.to_numpy().tolist()

In [ ]:
train_generator = TimeseriesGenerator(data=x_train, targets=y_train, length=17)
test_generator = TimeseriesGenerator(data=x_test, targets=y_test, length=17)

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
model = keras.models.Sequential([
    keras.layers.LSTM(512, input_shape = (17, 1), dropout=0.2, return_sequences=True,kernel_initializer='he_normal'),
    keras.layers.LSTM(512, dropout=0.2, return_sequences=True,kernel_initializer='he_normal'),
    keras.layers.LSTM(512, dropout=0.2, return_sequences=True,kernel_initializer='he_normal'),
    keras.layers.LSTM(512, dropout=0.2, return_sequences=True,kernel_initializer='he_normal'),
    keras.layers.LSTM(512, dropout=0.2, return_sequences=True,kernel_initializer='he_normal'),
    keras.layers.Dense(1)
])

In [ ]:
adam_optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=adam_optimizer, loss = 'mean_squared_error')
history = model.fit(train_generator, validation_data = test_generator, epochs = 10,batch_size=32)

In [ ]:
predictions = model.predict(test_generator)

In [ ]:
predictions

In [ ]:
y_test

In [ ]:
from sklearn.metrics import mean_squared_error
# print(mean_squared_error(predictions,y_test))
